In [24]:
import os
import subprocess
import numpy as np
import pandas as pd
import random
import mplcursors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager

In [25]:
def run_cpp_program(noise, nPose):
    result = subprocess.run(['../build/MyGTSAM', f'{noise}', f'{nPose}'], 
                            capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running C++ program: {result.stderr}")
        return np.inf

    return result.stdout.strip().split('\n')

In [26]:
content = pd.read_csv('pose_GT_8.csv', header=None)
pose_GT = content.values
pose_GT = np.delete(pose_GT, 2, axis=1)

content = pd.read_csv("/home/ali/Github/Kamal_Dataset/seq8/encoder/encoder.csv", header=None)
Data = content.values
delta = Data[:,0]
tsta = Data[:,1]
brk = Data[:,2]

content = pd.read_csv('initial_estimates_8.csv', header=None)
initialEst = content.values

nPose = len(initialEst)-8

cable_GT = initialEst[nPose:nPose+4,0].copy()
anchor_GT = initialEst[nPose+4:,:].copy()

# Perturbation

In [27]:
result = []
pose = []
anchor_STD = 0.3
cable_prior_sigma = 0.1
file_path = 'perturb.csv'
file_exists = os.path.isfile(file_path)
for i in range(250):
    initialEst[nPose+4,:] = np.random.normal(anchor_GT[0], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 1
    initialEst[nPose+5,:] = np.random.normal(anchor_GT[1], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 2
    initialEst[nPose+6,:] = np.random.normal(anchor_GT[2], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 3
    initialEst[nPose+7,:] = np.random.normal(anchor_GT[3], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 4
    
    initialEst_pd = pd.DataFrame(initialEst)
    initialEst_pd.to_csv('../dataset/estimates_8.csv', header=False, index=False)
    Opt = list(map(float, run_cpp_program(cable_prior_sigma, 31)))
    values = np.concatenate((np.array((initialEst[nPose-1,0], initialEst[nPose,0], initialEst[nPose+1,0], initialEst[nPose+2,0], initialEst[nPose+3,0])),
                             initialEst[nPose+4], initialEst[nPose+5], initialEst[nPose+6], initialEst[nPose+7], Opt[:15]))
    result.append(values)
    pose.append(Opt[13:])

result = np.array(result)
pose = np.array(pose)
result_pd = pd.DataFrame(result)
result_pd.columns = ['s0', 'm00', 'n00', 'k00', 'h00', 'A10_x', 'A10_y', 'A20_x', 'A20_y', 'A30_x', 'A30_y', 'A40_x', 'A40_y',
                     's', 'm0', 'n0', 'k0', 'h0', 'A1_x', 'A1_y', 'A2_x', 'A2_y', 'A3_x', 'A3_y', 'A4_x', 'A4_y', 'X0_x', 'X0_y']

In [29]:
plt.figure()
%matplotlib qt

plt.scatter(result_pd['A10_x'], result_pd['A10_y'], color="tab:cyan", label="Anchor 1 Initial Values")
plt.scatter(result_pd['A20_x'], result_pd['A20_y'], color="tab:pink", label="Anchor 2 Initial Values")
plt.scatter(result_pd['A30_x'], result_pd['A30_y'], color="gold", label="Anchor 3 Initial Values")
plt.scatter(result_pd['A40_x'], result_pd['A40_y'], color="limegreen", label="Anchor 4 Initial Values")

mean = np.mean(np.column_stack((result_pd['A1_x'], result_pd['A1_y'])), axis=0)
plt.scatter(result_pd['A1_x'], result_pd['A1_y'], label=f'Anchor 1 Optimization Result', color="tab:blue")

mean = np.mean(np.column_stack((result_pd['A2_x'], result_pd['A2_y'])), axis=0)
plt.scatter(result_pd['A2_x'], result_pd['A2_y'], label=f'Anchor 2 Optimization Result', color="purple")

mean = np.mean(np.column_stack((result_pd['A3_x'], result_pd['A3_y'])), axis=0)
plt.scatter(result_pd['A3_x'], result_pd['A3_y'], label=f'Anchor 3 Optimization Result', color="tab:orange")

mean = np.mean(np.column_stack((result_pd['A4_x'], result_pd['A4_y'])), axis=0)
plt.scatter(result_pd['A4_x'], result_pd['A4_y'], label=f'Anchor 4 Optimization Result', color="green")


plt.axis('equal')
csfont = {'fontname':'Times New Roman', 'fontsize': 22}
font = font_manager.FontProperties(family='Times New Roman',
                                   style='normal', size=18)
plt.xlabel("x (m)", **csfont)
plt.ylabel("y (m)", **csfont)

plt.tick_params(axis='both', which='major', labelsize=18, labelfontfamily='Times New Roman')
mplcursors.cursor(multiple=True)